# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
RANDOM_STATE = 1337228

In [3]:
data = pd.read_csv('datasets/users_behavior.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Целевым признаком** в данной работе выступает столбец **'is_ultra'**, признак **категориальный**, соответственно, поставлена задача **классификации**.

## Разбейте данные на выборки

In [4]:
#разбитие данных на выборки
train_data, split_data = train_test_split(data, test_size=0.40, random_state=RANDOM_STATE)
valid_data, test_data = train_test_split(split_data, test_size=0.50, random_state=RANDOM_STATE)
#создание таблиц с признаками
train_data_features = train_data.drop(['is_ultra'], axis=1)
train_data_target = train_data['is_ultra']
valid_data_features = valid_data.drop(['is_ultra'], axis=1)
valid_data_target = valid_data['is_ultra']
test_data_features = test_data.drop(['is_ultra'], axis=1)
test_data_target = test_data['is_ultra']

Так как у нас нет спрятанной тестовой выборки, данные разбивались по принципу 60/20/20:
- 60, обучающая выборка (train_data)
- 20, валидационная выборка (valid_data)
- 20, тестовая выборка (test_data)

## Исследуйте модели

Исследованию подлежат три модели:
1. Решающее дерево
2. Случайный лес
3. Логистическаая регрессия

### Решающее дерево

In [5]:
best_depth = 0
best_accuracy = 0

for depth in range(1,50):
    model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(valid_data_features)
    accuracy = accuracy_score(valid_data_target, valid_predictions)
    if accuracy > best_accuracy:
        best_depth = depth
        best_accuracy = accuracy

print('Лучшая глубина дерева:', best_depth)
print('Лучшая точность:', best_accuracy)

Лучшая глубина дерева: 8
Лучшая точность: 0.7978227060653188


Рассмотрим критерии Джини и энтропийный

In [6]:
best_tree_model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=8,criterion="gini")
best_tree_model.fit(train_data_features, train_data_target)
valid_predictions = best_tree_model.predict(valid_data_features)
accuracy = accuracy_score(valid_data_target, valid_predictions)
print('Глубина дерева: 8')
print('Точность:', accuracy)

Глубина дерева: 8
Точность: 0.7978227060653188


In [7]:
best_tree_model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=8,criterion="entropy")
best_tree_model.fit(train_data_features, train_data_target)
valid_predictions = best_tree_model.predict(valid_data_features)
accuracy = accuracy_score(valid_data_target, valid_predictions)
print('Глубина дерева: 8')
print('Точность:', accuracy)

Глубина дерева: 8
Точность: 0.80248833592535


**Энтропийный критерий даёт большую точность**

### Случайный лес

In [8]:
best_est = 0
best_accuracy = 0

for est in range(1,50):
    model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=est)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(valid_data_features)
    accuracy = accuracy_score(valid_data_target, valid_predictions)
    if accuracy > best_accuracy:
        best_est = est
        best_accuracy = accuracy

print('Лучшее количество деревьев', best_est)
print('Лучшая точность', best_accuracy)   

Лучшее количество деревьев 32
Лучшая точность 0.80248833592535


In [9]:
best_forest_model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=32, criterion="gini")
best_forest_model.fit(train_data_features, train_data_target)
valid_predictions = best_forest_model.predict(valid_data_features)
accuracy = accuracy_score(valid_data_target, valid_predictions)
print('Количество деревьев: 32')
print('Точность:', accuracy)   

Количество деревьев: 32
Точность: 0.80248833592535


In [10]:
best_forest_model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=32, criterion="entropy")
best_forest_model.fit(train_data_features, train_data_target)
valid_predictions = best_forest_model.predict(valid_data_features)
accuracy = accuracy_score(valid_data_target, valid_predictions)
print('Количество деревьев: 32')
print('Точность:', accuracy)   

Количество деревьев: 32
Точность: 0.7962674961119751


**Критерий Джини даёт большую точность**

### Логистическая регрессия

In [11]:
best_max_iter = 0
best_accuracy = 0

for max_iter in range(1,50):
    model = LogisticRegression(random_state=RANDOM_STATE, max_iter=max_iter)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(valid_data_features)
    accuracy = accuracy_score(valid_data_target, valid_predictions)
    if accuracy > best_accuracy:
        best_max_iter = max_iter
        best_accuracy = accuracy

print('Лучшое количество итерраций', best_max_iter)
print('Лучшая точность', best_accuracy)

C:\Users\dange\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dange\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\dange\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dange\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Лучшое количество итерраций 42
Лучшая точность 0.7060653188180405


C:\Users\dange\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dange\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [12]:
best_lr_model = LogisticRegression(random_state=RANDOM_STATE, max_iter = 42)
best_lr_model.fit(train_data_features, train_data_target)
valid_predictions = best_lr_model.predict(valid_data_features)
accuracy = accuracy_score(valid_data_target, valid_predictions)
print('Количество итерраций: 42')
print('Точность',accuracy)

Количество итерраций: 42
Точность 0.7060653188180405


C:\Users\dange\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Вывод

Модели решающего дерева и случайного леса показали одинаковую точность, в то время как модель логичстической регрессии показала худшую точность из всех вариантов моделей с разными гиперпараметрами. Теперь посмотрим как модели поведут себя на тестовой выборке

## Проверьте модель на тестовой выборке

In [13]:
test_predictions = best_tree_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Точность модели решающего дерева',accuracy)

test_predictions = best_forest_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Точность модели случайного леса',accuracy)

test_predictions = best_lr_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Точность модели логистической регресиии',accuracy)

Точность модели решающего дерева 0.7620528771384136
Точность модели случайного леса 0.7682737169517885
Точность модели логистической регресиии 0.6951788491446346


На тестовой выборке лучше остальных показала себя модель случайного леса со значением точности в 0.768.

**Исходя из результатов исследования, лучшей моделью с наивысшим значением точности является модель случайного леса с количеством деревьев в 32 и критерием Джини**

## (бонус) Проверьте модели на адекватность

Адекватность модели оценивается по нескольким параметрам. Для модели классификации это accuracy, precision и recall. Качество модели лучше всего отражают precision и recall и эти метрики складываются в F-мерe как единую оценку модели.

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [15]:
test_predictions = best_forest_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
precision = precision_score(test_data_target, test_predictions)
recall = recall_score(test_data_target, test_predictions)
f1 = f1_score(test_data_target, test_predictions)

print('Accuracy =', accuracy, 'Precision =', precision, 'Recall =', recall, 'f1 =', f1)

Accuracy = 0.7682737169517885 Precision = 0.6818181818181818 Recall = 0.5121951219512195 f1 = 0.584958217270195


Модель среднего качества